Imports & Configuration

In [3]:
# Imports
import os
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration
FOLDS_DIR    = "cv_folds"
FEATURE_COLS = [
    'PTS_8_game_avg','REB_8_game_avg','AST_8_game_avg',
    'PTS_8_game_slope','REB_8_game_slope','AST_8_game_slope',
    'opp_E_OFF_RATING','opp_E_DEF_RATING','opp_E_NET_RATING','opp_REB_PCT'
]
TARGET_COLS = ['PTS','REB','AST']
N_JOBS      = 4


Metrics

In [5]:
def compute_metrics(y_true: pd.DataFrame, y_pred: np.ndarray) -> dict:
    out = {}
    y_true_flat = y_true.values.flatten()
    y_pred_flat = y_pred.flatten()
    mse = mean_squared_error(y_true_flat, y_pred_flat)
    out['rmse_overall'] = np.sqrt(mse)
    out['mae_overall']  = mean_absolute_error(y_true_flat, y_pred_flat)
    out['r2_overall']   = r2_score(y_true_flat, y_pred_flat)

    for i, tgt in enumerate(TARGET_COLS):
        yt, yp = y_true.iloc[:, i], y_pred[:, i]
        m = mean_squared_error(yt, yp)
        out[f'rmse_{tgt}'] = np.sqrt(m)
        out[f'mae_{tgt}']  = mean_absolute_error(yt, yp)
        out[f'r2_{tgt}']   = r2_score(yt, yp)
    return out


Define per-fold runners

In [ ]:
from sklearn.impute import SimpleImputer

def run_knn(k, w, fold):
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold}_train.csv")
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold}_test.csv")
    X_tr, X_te = tr[FEATURE_COLS], te[FEATURE_COLS]
    y_tr, y_te = tr[TARGET_COLS], te[TARGET_COLS]

    # Impute missing values
    imp = SimpleImputer(strategy='mean')
    X_tr = imp.fit_transform(X_tr)
    X_te = imp.transform(X_te)

    # Scale
    scaler = StandardScaler().fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_te = scaler.transform(X_te)

    # Fit & predict
    model = KNeighborsRegressor(n_neighbors=k, weights=w)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)

    # Metrics
    res = compute_metrics(y_te, y_pred)
    res.update({'n_neighbors':k, 'weights':w, 'fold':fold})
    return res

def run_svr(C, g, ker, fold):
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold}_train.csv")
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold}_test.csv")
    X_tr, X_te = tr[FEATURE_COLS], te[FEATURE_COLS]
    y_tr, y_te = tr[TARGET_COLS], te[TARGET_COLS]

    # Impute missing values
    imp = SimpleImputer(strategy='mean')
    X_tr = imp.fit_transform(X_tr)
    X_te = imp.transform(X_te)

    # Scale
    scaler = StandardScaler().fit(X_tr)
    X_tr, X_te = scaler.transform(X_tr), scaler.transform(X_te)

    # Fit & predict
    base = SVR(C=C, gamma=g, kernel=ker)
    model = MultiOutputRegressor(base)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)

    # Metrics
    res = compute_metrics(y_te, y_pred)
    res.update({'C':C, 'gamma':g, 'kernel':ker, 'fold':fold})
    return res

def run_mlp(hls, a, lr, fold):
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold}_train.csv")
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold}_test.csv")
    X_tr, X_te = tr[FEATURE_COLS], te[FEATURE_COLS]
    y_tr, y_te = tr[TARGET_COLS], te[TARGET_COLS]

    # Impute missing values
    imp = SimpleImputer(strategy='mean')
    X_tr = imp.fit_transform(X_tr)
    X_te = imp.transform(X_te)

    # Scale
    scaler = StandardScaler().fit(X_tr)
    X_tr, X_te = scaler.transform(X_tr), scaler.transform(X_te)

    # Fit & predict
    base = MLPRegressor(hidden_layer_sizes=hls, alpha=a,
                        learning_rate_init=lr, max_iter=1000,
                        early_stopping=True, n_iter_no_change=20)
    model = MultiOutputRegressor(base)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)

    # Metrics
    res = compute_metrics(y_te, y_pred)
    res.update({'hidden_layer_sizes':hls,'alpha':a,'lr':lr,'fold':fold})
    return res



Build the grids and launch

In [ ]:
from tqdm import tqdm

# k-NN grid (4 k values, 2 weightings, and 10 folds to do 80 runs)
knn_results = []
for k, w, f in tqdm(knn_params, desc="k-NN grid"):
    knn_results.append(run_knn(k, w, f))
knn_df = pd.DataFrame(knn_results)

# SVR grid (3 C values, 2 gammas, and 10 folds to do 60 runs)
svr_results = []
for C, g, ker, f in tqdm(svr_params, desc="SVR grid"):
    svr_results.append(run_svr(C, g, ker, f))
svr_df = pd.DataFrame(svr_results)

# MLP grid (3 layer-configs, 2 alphas, 2 lrs, and 10 folds to do 120 runs)
mlp_results = []
for h, a, lr, f in tqdm(mlp_params, desc="MLP grid"):
    mlp_results.append(run_mlp(h, a, lr, f))
mlp_df = pd.DataFrame(mlp_results)



MLP grid:   8%|▊         | 10/120 [00:18<04:30,  2.46s/it]c:\Users\aweso\OneDrive\Documents\VSCode\NBA_Player_Props-1\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
MLP grid:  25%|██▌       | 30/120 [01:43<03:49,  2.55s/it]c:\Users\aweso\OneDrive\Documents\VSCode\NBA_Player_Props-1\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
MLP grid:  58%|█████▊    | 70/120 [05:06<02:24,  2.89s/it]c:\Users\aweso\OneDrive\Documents\VSCode\NBA_Player_Props-1\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
MLP 

Export

In [19]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print("k-NN results (all runs):")
print(knn_df, "\n")

print("SVR results (all runs):")
print(svr_df, "\n")

print("MLP results (all runs):")
print(mlp_df, "\n")

print("Shapes:")
print(f"  k-NN: {knn_df.shape}")
print(f"  SVR : {svr_df.shape}")
print(f"  MLP : {mlp_df.shape}\n")

print("Folds present:")
print(f"  k-NN folds: {sorted(knn_df['fold'].unique())}")
print(f"  SVR  folds: {sorted(svr_df['fold'].unique())}")
print(f"  MLP  folds: {sorted(mlp_df['fold'].unique())}\n")

knn_df.to_csv("knn_metrics.csv", index=False)
svr_df.to_csv("svr_metrics.csv", index=False)
mlp_df.to_csv("mlp_metrics.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')



k-NN results (all runs):
    rmse_overall  mae_overall  r2_overall  rmse_PTS   mae_PTS    r2_PTS  \
0       5.770450     3.983960    0.415836  8.990116  7.104768  0.021894   
1       5.265830     3.592201    0.496040  8.157849  6.423149  0.122219   
2       5.107113     3.493523    0.494242  7.920012  6.239810  0.129430   
3       5.144671     3.499028    0.527600  7.969717  6.268333  0.202230   
4       5.151920     3.513889    0.524876  7.974022  6.287415  0.176983   
5       5.170258     3.526008    0.549529  7.981425  6.255960  0.226208   
6       5.124336     3.515390    0.519427  7.907255  6.202543  0.180726   
7       5.304879     3.560256    0.525036  8.225243  6.290385  0.182872   
8       5.047020     3.492343    0.514548  7.777757  6.160244  0.133928   
9       5.332874     3.613534    0.507029  8.254618  6.406802  0.150043   
10      5.774432     3.978676    0.415029  9.009114  7.112440  0.017756   
11      5.270749     3.599077    0.495098  8.161977  6.425945  0.121330   
